In [ ]:
import weave
from weave.monitoring import StreamTable
from weave.stream_data_interfaces import TraceSpanDict

In [ ]:
st = StreamTable("timssweeney/monitor_dev_test/syn_mon_9")

In [ ]:
import random
from uuid import uuid4
import datetime
import json

names = [
"fit",
"predict",
"transform",
"compile",
"evaluate",
"GridSearchCV",
"train_test_split",
"Sequential",
"early_stopping",
"OneHotEncoder",
"Dense",
"MinMaxScaler",
"dropout",
"confusion_matrix",
"lstm",
]

def simple_dict():
    return {
        "a_1": 42,
        "a_2": "Test",
        "a_3": True,
        "a_4": {
            "_kind": "AGENT",
            "list": [1,2,3],
            "dict": {
                "hello": "world"
            }
        }
    }

def create_random_span(accumulator, trace_id=None, parent_id=None, start_time_s=None):
    name = random.choice(names)
    kind = random.choice([None, 'CHAIN','AGENT','TOOL','LLM',])
    
    status_code = "UNSET"
    status = random.randint(0, 10000)
    if status == 0:
        status_code = "ERROR"
    elif status <= 9995:
        status_code = "SUCCESS"
    exception = random.choice([None, "Error"]) if status_code == "ERROR" else None
    span_id = str(uuid4())
    if trace_id is None:
        trace_id = str(uuid4())
    attributes = simple_dict()

    # Special Attributes
    attributes["span_kind"] = kind
    attributes["model"] = {
        "id": "4edfghu7654edfg",
        "obj": json.dumps(simple_dict())
    }
    inputs = simple_dict()
    output = simple_dict()
    summary = simple_dict()
    
    if start_time_s is None:
        start_time_s = datetime.datetime.now().timestamp()

    if parent_id is None:
        num_children = random.choice([1,2,3,4,5])
    else:
        num_children = random.choice([0,0,0,0,0,1,2])

    if num_children == 0 or status_code == 'ERROR':
        duration_s = random.randint(1, 10000) / 1000
    else:
        duration_s = 0
        for i in range(num_children):
            child_span = create_random_span(accumulator, trace_id, span_id, duration_s + start_time_s)
            duration_s += child_span['end_time_s'] - child_span['start_time_s']
            if child_span['status_code'] == 'ERROR':
                status_code = 'ERROR'
                exception = child_span['exception']
                break
        
    end_time_s = start_time_s + duration_s
    self_span = TraceSpanDict(
        name=name,
        span_id=span_id,
        trace_id=trace_id,
        status_code=status_code,
        start_time_s=start_time_s,
        end_time_s=end_time_s,
        parent_id=parent_id,
        attributes=attributes,
        inputs=inputs,
        output=output,
        summary=summary,
        exception=exception,
    )
    # Purely a simulation hack
    self_span["timestamp"] = datetime.datetime.fromtimestamp(start_time_s)
    accumulator.append(self_span)
    return self_span

In [ ]:
start_time_s = datetime.datetime.now().timestamp()
for i in range(10):
    acc = []
    create_random_span(acc, start_time_s=start_time_s)
    st.log(acc)
    start_time_s = max([a['timestamp'].timestamp() for a in acc])

st.finish()

In [ ]:
acc = []
start_time_s = datetime.datetime.now().timestamp()
create_random_span(acc, start_time_s=start_time_s)
weave.show(weave.panels.panel_trace.Trace(acc))

In [ ]:
backup_acc = [{'name': 'lstm',
  'span_id': '5a2038a2-6fae-4ded-aad8-3930f915ddb3',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911275.188327,
  'end_time_s': 1694911284.752327,
  'parent_id': '7789170f-5621-4f6f-bdce-c56306535c58',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'TOOL',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 15, 188327)},
 {'name': 'lstm',
  'span_id': '7789170f-5621-4f6f-bdce-c56306535c58',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911275.188327,
  'end_time_s': 1694911284.752327,
  'parent_id': '00407abc-964c-401d-8947-c614064feb58',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'LLM',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 15, 188327)},
 {'name': 'early_stopping',
  'span_id': '0d7023dc-d709-42ed-88bf-88d083a270fa',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911284.752327,
  'end_time_s': 1694911286.387327,
  'parent_id': '00407abc-964c-401d-8947-c614064feb58',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'LLM',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 24, 752327)},
 {'name': 'transform',
  'span_id': '00407abc-964c-401d-8947-c614064feb58',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911275.188327,
  'end_time_s': 1694911286.387327,
  'parent_id': 'bb9c9027-87c9-49fc-90d0-7b58d946ff42',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'LLM',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 15, 188327)},
 {'name': 'predict',
  'span_id': '4325f753-4332-4e71-a44e-30519de5dc7e',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911286.387327,
  'end_time_s': 1694911292.105327,
  'parent_id': 'd5ba8785-0584-4910-8b7b-eb1f9ae7fdf1',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'AGENT',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 26, 387327)},
 {'name': 'MinMaxScaler',
  'span_id': 'd5ba8785-0584-4910-8b7b-eb1f9ae7fdf1',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911286.387327,
  'end_time_s': 1694911292.105327,
  'parent_id': 'bb9c9027-87c9-49fc-90d0-7b58d946ff42',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'TOOL',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 26, 387327)},
 {'name': 'evaluate',
  'span_id': '5c8857bb-0aaa-4a14-b69a-ff941028b155',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911292.105327,
  'end_time_s': 1694911300.713327,
  'parent_id': 'a1b7d19c-d846-435f-9515-b3cab0e8fcec',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'TOOL',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 32, 105327)},
 {'name': 'transform',
  'span_id': 'a1b7d19c-d846-435f-9515-b3cab0e8fcec',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911292.105327,
  'end_time_s': 1694911300.713327,
  'parent_id': '1f1b53c5-d9f3-46f6-814d-6276292c1fd0',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'LLM',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 32, 105327)},
 {'name': 'evaluate',
  'span_id': '1f1b53c5-d9f3-46f6-814d-6276292c1fd0',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911292.105327,
  'end_time_s': 1694911300.713327,
  'parent_id': 'e0b418bc-c960-456b-b020-5626c51ae1a9',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'AGENT',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 32, 105327)},
 {'name': 'MinMaxScaler',
  'span_id': '623b430b-123b-420e-918a-7295eee242a4',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911300.713327,
  'end_time_s': 1694911304.686327,
  'parent_id': 'e0b418bc-c960-456b-b020-5626c51ae1a9',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'TOOL',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 40, 713327)},
 {'name': 'compile',
  'span_id': 'e0b418bc-c960-456b-b020-5626c51ae1a9',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911292.105327,
  'end_time_s': 1694911304.686327,
  'parent_id': 'bb9c9027-87c9-49fc-90d0-7b58d946ff42',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'LLM',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 32, 105327)},
 {'name': 'early_stopping',
  'span_id': 'deed3315-a139-4656-b3a4-8b71396961fd',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911304.686327,
  'end_time_s': 1694911312.653327,
  'parent_id': 'bb9c9027-87c9-49fc-90d0-7b58d946ff42',
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'LLM',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 44, 686327)},
 {'name': 'train_test_split',
  'span_id': 'bb9c9027-87c9-49fc-90d0-7b58d946ff42',
  'trace_id': 'c3370313-2bbf-42f1-918d-a7930b1982de',
  'status_code': 'SUCCESS',
  'start_time_s': 1694911275.188327,
  'end_time_s': 1694911312.653327,
  'parent_id': None,
  'attributes': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}},
   'span_kind': 'CHAIN',
   'model': {'id': '4edfghu7654edfg',
    'obj': '{"a_1": 42, "a_2": "Test", "a_3": true, "a_4": {"_kind": "AGENT", "list": [1, 2, 3], "dict": {"hello": "world"}}}'}},
  'inputs': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'output': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'summary': {'a_1': 42,
   'a_2': 'Test',
   'a_3': True,
   'a_4': {'_kind': 'AGENT', 'list': [1, 2, 3], 'dict': {'hello': 'world'}}},
  'exception': None,
  'timestamp': datetime.datetime(2023, 9, 16, 17, 41, 15, 188327)}]